## Setup

In [ ]:
import os
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

SEED = 1234
tf.random.set_seed(SEED)
np.random.seed(SEED)

os.chdir("/content")
cwd = os.getcwd()

drive_path = os.path.join(cwd, 'drive')
my_drive_path = os.path.join(drive_path, 'My Drive')

project_path = os.path.join(my_drive_path, 'AN2DL-Project')

data_path = os.path.join(project_path, 'data')
data_path2 = os.path.join(my_drive_path, 'dataset')
train_path = os.path.join(data_path, 'Training')
train_path2 = os.path.join(data_path2, 'Training')
test_path = os.path.join(data_path, 'Test_Dev')
logs_path = os.path.join(project_path, 'logs')
results_path = os.path.join(project_path, 'results')

X_dir = 'Images'
Y_dir = 'Masks'

teams = ['Weedelec']
crops = ['Mais']


train_imgs_path = os.path.join(train_path2, 'training.txt')
valid_imgs_path = os.path.join(train_path2, 'validation.txt')

In [ ]:
from google.colab import drive
drive.mount(drive_path)

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
img_h = 400
img_w = 400


def start_points(size, split_size, overlap=0.1):
    points = [0]
    stride = int(split_size * (1-overlap))
    counter = 1
    while True:
        pt = stride * counter
        if pt + split_size >= size:
            points.append(size - split_size)
            break
        else:
            points.append(pt)
        counter += 1
    return points

def tile_image(img, split_width=img_w,split_height=img_h, overlap=0):

    
    img_width, img_height = img.size
    #img = img.resize((img_width, img_height))

    X_points = start_points(img_width, split_width, overlap)
    Y_points = start_points(img_height, split_height, overlap)

    img_arr = np.array(img)

    count = 0

    img_tiles = []
    for i in Y_points:
        for j in X_points:
            split = img_arr[i:i+split_height, j:j+split_width]
            img_tiles.append(split)
            #plt.imshow(split)
            #plt.show()

            count += 1
    return np.asarray(img_tiles)

In [ ]:


for team in teams:
  for crop in crops:
    print(team+"..."+crop+"...")

    for root,dirs,files in os.walk(os.path.join(train_path, team, crop, X_dir)):
      print("Found {} images for team {}, crop {}".format(len(files),team,crop))
      for file in files:
        if team == 'Roseau':
          img = Image.open(os.path.join(train_path, team, crop, X_dir, os.path.splitext(file)[0] + '.png'))
        else:
          img = Image.open(os.path.join(train_path, team, crop, X_dir, os.path.splitext(file)[0] + '.jpg'))
        mask = Image.open(os.path.join(train_path, team, crop, Y_dir, os.path.splitext(file)[0] + '.png'))

        tiles_image = tile_image(img, img_w, img_h,0.3)
        tiles_mask = tile_image(mask,img_w, img_h,0.3)

        for x in range(len(tiles_image)):
          imm = Image.fromarray(tiles_image[x])
          imm_dir = os.path.join(train_path2, team, crop, X_dir)
          os.chdir(my_drive_path)
         
          if team == 'Roseau':
            imm.save(os.path.join(imm_dir,os.path.splitext(file)[0] +'_'+str((x+1))+ '.png'),"PNG")
          else:
            imm.save(os.path.join(imm_dir,os.path.splitext(file)[0] +'_'+str((x+1))+ '.jpg'))

          mask_dir = os.path.join(train_path2, team, crop, Y_dir)
          maskm =Image.fromarray(tiles_mask[x])
          #os.chdir(mask_dir)
          
          maskm.save(os.path.join(mask_dir,os.path.splitext(file)[0] +'_'+str((x+1))+ '.png'),"PNG")
